(case_wpcc)=
## wPCC tests

In [ ]:
# %load imports.py
%load_ext autoreload
%autoreload 2
%reload_kedro
%config Completer.use_jedi = False  ## (To fix autocomplete)
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
from src.models.vmm import ModelSimulator
import matplotlib.pyplot as plt
from src.visualization.plot import track_plots, plot, captive_plot
import kedro
import numpy as np
import os.path
import anyconfig

import matplotlib
plt.style.use('paper')
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('pdf')

from src.symbols import *

# Read configs:
conf_path = os.path.join("../../conf/base/")
runs_globals_path = os.path.join(
    conf_path,
    "runs_globals.yml",
)

runs_globals = anyconfig.load(runs_globals_path)
model_test_ids = runs_globals["model_test_ids"]

join_globals_path = os.path.join(
    conf_path,
    "join_globals.yml",
)

joins = runs_globals["joins"]
join_runs_dict = anyconfig.load(join_globals_path)

globals_path = os.path.join(
    conf_path,
    "globals.yml",
)
global_variables = anyconfig.load(globals_path)



vmm_names = global_variables["vmms"]

from wPCC_pipeline.pipelines.motion_regression.nodes import predict_force, fit_motions, create_model_from_motion_regression
from wPCC_pipeline.pipelines.prediction.nodes import simulate_euler
from src.prime_system import PrimeSystem

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression
from sklearn.pipeline import Pipeline
from src.parameters import df_parameters
p = df_parameters['symbol']
from src.feature_selection import drop_multicollinearity
from src.bias_variance_tradeoff import train_test_split_run
from sklearn.metrics import r2_score
from wPCC_pipeline.pipelines.motion_regression.nodes import fit_motions, create_model_from_motion_regression
from src.models.regression import Regression
from myst_nb import glue
import src.symbols as s
from IPython.display import Latex
from src.substitute_dynamic_symbols import run
import seaborn as sns
from src.feature_selection import feature_imporance, BestFeatures, DropCorrelation
from wPCC_pipeline.turning_circle import TurningCircle
from jb_helpers import parameter_to_latex, df_to_myst
from wPCC_pipeline.monte_carlo import monte_carlo, parameter_variation

In [ ]:
ship="wpcc"
#vmm_name = "vmm_martins_simple"
vmm_name = "vmm_abkowitz"
vmm = catalog.load(vmm_name)

ship_data = catalog.load(f"{ship}.ship_data")

#regression = catalog.load(f"{ship}.updated.{vmm_name}.joined.regression")
#regression.diff_eq_Y.exclude_parameters.pop('Ydelta')
#regression.diff_eq_Y.exclude_parameters.pop('Ythrustdelta')


data = catalog.load(f"{ship}.updated.joined.data_ek_smooth")

added_masses = catalog.load(f"{ship}.added_masses")
exclude_parameters = catalog.load(f"params:{ship}.motion_regression.exclude_parameters")
columns = ['u','v','r','u1d','v1d','r1d','delta','thrust','id','x0','y0','psi']
data_with_force = predict_force(data=data[columns], added_masses=added_masses, ship_parameters=ship_data, vmm=vmm)

In [ ]:
def break_plot(df, **kwargs):
    
    df_ = df.copy()
    mask = np.concatenate([[False],np.diff(df_.index)>1])
    df_.loc[mask] = np.NaN
    df_.plot(**kwargs)


In [ ]:
ids = list(data_with_force['id'].unique())
ids_train = ids.copy()
id_test = 22774
ids_train.remove(id_test)
mask = data_with_force['id'].isin(ids_train)
data_train = data_with_force.loc[mask].copy()
assert not id_test in data_train['id'].unique()

ps = PrimeSystem(**ship_data)
data_prime = ps.prime(data_train, U=data_with_force['U'])

In [ ]:
ids_test = [
    22771,
    22772,
    22773,
]

mask = data_train['id'].isin(ids_test)
data_validation = data_train.loc[mask].copy()
data_sub_train = data_train.loc[~mask].copy()
mask = data['id']==id_test
data_testing = data_with_force.loc[mask]

In [ ]:
def rotate(df):
    
    dpsi = 2*np.pi*np.random.random()
    
    df['psi']+=dpsi
    x0 = df['x0'].copy()
    y0 = df['y0'].copy()
    
    df['x0'] = np.cos(dpsi)*x0 - np.sin(dpsi)*y0
    df['y0'] = np.sin(dpsi)*x0 + np.cos(dpsi)*y0
        
    
    return df
    
    

In [ ]:
np.random.seed(3)
fig,ax=plt.subplots()

dataframes = {id:rotate(df_) for id, df_ in data_sub_train.groupby(by='id')}
styles = {id:{'style':'b-','label':'_nolegend_'} for id, df_ in data_sub_train.groupby(by='id')}

dataframes.update({id:rotate(df_) for id, df_ in data_validation.groupby(by='id')})
styles.update({id:{'style':'g-','label':'_nolegend_'} for id, df_ in data_validation.groupby(by='id')})

dataframes.update({id:rotate(df_) for id, df_ in data_testing.groupby(by='id')})
styles.update({id:{'style':'r-','label':'_nolegend_'} for id, df_ in data_testing.groupby(by='id')})

track_plots(dataframes, lpp=ship_data['L'], beam=ship_data['B'],  styles=styles, N=2, ax=ax);

ax.plot([],'b-',label='Training')
ax.plot([],'g-',label='Validation')
ax.plot([],'r-',label='Testing')


ax.legend();
glue('fig_traintest',fig, display=False)

In [ ]:
ps = PrimeSystem(**ship_data)
regression = Regression(
        vmm=vmm,
        data=data_train,
        added_masses=added_masses,
        ship_parameters=ship_data,
        prime_system=ps,
        exclude_parameters=exclude_parameters, 
        connect_equations_Y_N_rudder=True
    )

In [ ]:
Xs = {}
ys = {}


Xs['X'], ys['X'] = regression.diff_eq_X.calculate_features_and_label(data=data_prime, y=data_prime['fx'])
Xs['Y'], ys['Y'] = regression.diff_eq_Y.calculate_features_and_label(data=data_prime, y=data_prime['fy'])
Xs['N'], ys['N'] = regression.diff_eq_N.calculate_features_and_label(data=data_prime, y=data_prime['mz'])

dofs = list(Xs.keys())

In [ ]:
df_parameters = pd.DataFrame()
df_parameters['mean'] = regression.model_N.params
df_parameters['std'] = regression.model_N.bse
df_parameters.sort_values(by='std', ascending=False, inplace=True)

In [ ]:
df_table = df_parameters.iloc[0:5]
glue('Ndelta',int(df_parameters.loc['Ndelta','mean'].round(0)))
glue('eNdelta',int(df_parameters.loc['Ndelta','std'].round(0)))

glue('Nvvdelta',int(df_parameters.loc['Nvvdelta','mean'].round(0)))
glue('eNvvdelta',int(df_parameters.loc['Nvvdelta','std'].round(0)))

df_table

Large problems with multicolliearity was encountered when applying the PIT on the wPCC data with AVMM. The absolute correlation coefficient between the features in the wPCC yaw moment regression are shown in {numref}`fig_Ncorr`. It can be seen that most of the coefficients have very high absolute correlation (indicated in black).

```{glue:figure} fig_Ncorr
:figwidth: 1000px
:name: "fig_Ncorr"

Absolute correlation between the features in the wPCC yaw moment regression of AVMM
```

Some of the regressed hydrodynamic derivatives in the AVMM also have very large values and large uncertainty.









In [ ]:
X = Xs['N']
corr = X.corr().abs()
corr_ = np.tril(corr, k=-1)
corr_ = pd.DataFrame(corr_, index=X.columns, columns=X.columns)

fig,ax=plt.subplots()
grid = sns.heatmap(corr_, cmap='gray_r')
glue('fig_Ncorr',fig, display=False)
fig.savefig("figures/fig_Ncorr.pdf")

In [ ]:
ship_model = catalog.load(f"{ship}.updated.{vmm_name}.joined.model")

## VMM Martin

In [ ]:
vmm_martin = catalog.load("vmm_martins_simple")
ek_martin = catalog.load(f"{ship}.vmm_martins_simple.ek")

In [ ]:
regression_martin, _ = fit_motions(data=data_sub_train, 
                                   added_masses=added_masses, 
                                   ship_data=ship_data, 
                                   vmm=vmm_martin, 
                                   exclude_parameters=exclude_parameters)

ship_model_martin = create_model_from_motion_regression(regression=regression_martin)
#result_martin = ship_model_martin.simulate(data_validation)
#df_predict_martin = result_martin.result.copy()

In [ ]:
regression_abkowitz, _ = fit_motions(data=data_sub_train, 
                                   added_masses=added_masses, 
                                   ship_data=ship_data, 
                                   vmm=vmm, 
                                   exclude_parameters=exclude_parameters)

ship_model_abkowitz = create_model_from_motion_regression(regression=regression_abkowitz)

The mean values and standard error (se) of the hydrodynamic derivatives for wPCC obtained with PIT applied on MAVMM ([eq](eqxmartinssimple),[eq](eqymartinssimple), [eq](eqnmartinssimple)) are shown in {numref}`wpcc_derivatives`. The standard error is much smaller for this model compared to AVMM.

```{list-table} wPCC MAVMM derivatives (times 1000)
:header-rows: 1
:name: wpcc_derivatives
* - name
  - mean
  - se
  - name
  - mean
  - se
  - name
  - mean
  - se
* - $ X_{\delta\delta} $
  - -2.927
  - 0.011
  - $ Y_{ur} $
  - -65.507
  - 0.082
  - $ N_{\delta} $
  - -1.993
  - 0.002
* - $ X_{vr} $
  - -7.737
  - 0.066
  - $ Y_{v} $
  - -20.347
  - 0.016
  - $ N_{T\delta} $
  - -5.392
  - 0.599
* - $ X_{rr} $
  - -1.413
  - 0.026
  - $ Y_{u} $
  - -0.027
  - 0.001
  - $ N_{r} $
  - -37.341
  - 0.096
* - $ X_{uu} $
  - 20.124
  - 0.137
  - $ Y_{r} $
  - 64.14
  - 0.083
  - $ N_{u} $
  - -0.003
  - 0.0
* - $ X_{u} $
  - -20.948
  - 0.137
  - 
  - 
  - 
  - $ N_{ur} $
  - 35.525
  - 0.096
* - 
  - 
  - 
  - 
  - 
  - 
  - $ N_{v} $
  - -0.05
  - 0.004
* - 
  - 
  - 
  - 
  - 
  - 
  - $ N_{vv\delta} $
  - -19.051
  - 0.054

```

Forces and moment predicted with the VMMs fitted on the training set are show in {numref}`fig_validation_forces`. It can be seen that AVMM overpredicts the forces by far due to the anticipated extrapolation error. Simulations of the validation cases where therefore only possible with the MAVMM as shown for one of the ZigZag20/20 validation cases in {numref}`fig_validation_sim`.

```{glue:figure} fig_validation_forces
:figwidth: 1000px
:name: "fig_validation_forces"

Validation of force models for wPCC ZigZag20/20.
```

```{glue:figure} fig_validation_sim
:figwidth: 1000px
:name: "fig_validation_sim"

Validation with simulations for wPCC ZigZag20/20.
```

In [ ]:
validation_martin = regression_martin.predict(data_validation)
validation_abkowitz = regression_abkowitz.predict(data_validation)

fig,axes=plt.subplots(nrows=3)

for ax,dof in zip(axes,['fx','fy','mz']):
    
    validation_abkowitz.plot(y=dof,ax=ax, label='Validation AVMM', style='r-')
    validation_martin.plot(y=dof,ax=ax, label='Validation MAVMM', style='b--')
    data_validation.plot(y=dof,ax=ax, label='True', style='k-.')
    
    ax.set_ylabel(dof)
    ax.set_xlabel('sample')
    ax.get_legend().set_visible(False)
    
axes[0].legend();
glue("fig_validation_forces",fig, display=False)
fig.savefig("figures/fig_validation_forces.pdf")

In [ ]:
for id_validation in ids_test[0:1]:
    
    df_test = catalog.load(f'{ship}.updated.{id_validation}.data_ek_smooth')
    result_martin = ship_model_martin.simulate(df_test)   
    
    dataframes = {'True':df_test,
                 'Validation MAVMM':result_martin.result,
                 }
    
    try:
        result_abkowitz = ship_model_abkowitz.simulate(df_test)
    except:
        pass
    else:
        dataframes['Abkowitz']=result_abkowitz.result
    
    track_plots(dataframes, lpp=ship_data['L'], beam=ship_data['B'],  N=2);
    fig = plot(dataframes=dataframes, keys=['u','v','r'], ncols=1, zero_origo=True);

glue("fig_validation_sim",fig, display=False)
fig.savefig("figures/fig_validation_sim.pdf")

In [ ]:
df_test = catalog.load(f'{ship}.updated.{id_test}.data_ek_smooth')
ek = catalog.load(f"{ship}.{vmm_name}.ek")

In [ ]:
#regression_martin, _ = fit_motions(data=data_train, 
#                                   added_masses=added_masses, 
#                                   ship_data=ship_data, 
#                                   vmm=vmm_martin, 
#                                   exclude_parameters=exclude_parameters)
#
#ship_model_martin = create_model_from_motion_regression(regression=regression_martin)
#result_martin = ship_model_martin.simulate(df_test) 

In [ ]:
#np.random.seed(42)
#N_=100
#
#df_parameter_variations = []
#
#for dof, rv in rvs.items():
#    mean = means[dof]
#    df_parameter_variations.append(pd.DataFrame(data=rv.rvs(N_), columns=mean.index))
#    
#df_parameter_variations = pd.concat(df_parameter_variations, axis=1)

In [ ]:
df_parameter_variations = parameter_variation(regression = regression_martin, N_=100, seed=42)

interesting = ['u','v','r','x0','y0','psi','delta','thrust','U']
data = df_test[interesting].copy()
try:
    loaders = catalog.load("wpcc.monte_carlo_simulation")
except Exception:
    # Slow way...
    dataframes_monte_carlo = monte_carlo(data, df_parameter_variations, model=ship_model_martin, fast=True, ek=ek)
    catalog.save("wpcc.monte_carlo_simulation", data={str(key):value for key,value in dataframes_monte_carlo.items()})
else:
    dataframes_monte_carlo = {key:loader() for key,loader in loaders.items()}

In [ ]:
dataframes = {
    'Model test' : df_test,
    'Test MAVMM' : result_martin.result,
        
}

styles = {
    'Model test' : {'style':'r--'},
    'Test MAVMM' : {'style':'k-'},
    
}

for index in dataframes_monte_carlo.keys():
    styles[index] = {'style':'b-', 
                     'alpha':0.1,
                     'label':'_Hidden label',
                     'zorder' : -100}

In [ ]:
fig,ax=plt.subplots()
track_plots(dataframes_monte_carlo, lpp=ship_data['L'], beam=ship_data['B'],  styles=styles, plot_boats=False, ax=ax);
track_plots(dataframes, lpp=ship_data['L'], beam=ship_data['B'],  styles=styles, N=5, ax=ax);
ax.plot([],[],'b-',label='Monte Carlo MAVMM')
ax.legend()

glue("fig_track_plot_testing_sim", fig, display=False)
fig.savefig("figures/fig_track_plot_testing_sim.pdf")

For wPCC the prediction was conducted using simulation with the MAVMM trained on the training and validation dataset. Result from the final prediction of the turning circle test is seen in {numref}`fig_track_plot_testing_sim`, {numref}`fig_testing_sim`. Monte Carlo simulations with alternative realizations of the regression are also shown in these figures. The alternative realizations are very similar to the model with mean values of the regression (black line), since the standard deviation of the regressed parameters are very small.

Advance and tactical diameter {cite:p}`imo_standards_2002` differs 4% and 1% between simulation with the VMM and corresponding results from the model tests {numref}`tab_wpcc_advance`.

```{glue:figure} fig_track_plot_testing_sim
:figwidth: 1000px
:name: "fig_track_plot_testing_sim"

Turning circle test case for wPCC, track plots from model test and simulation.
```

```{glue:figure} fig_testing_sim
:figwidth: 1000px
:name: "fig_testing_sim"

Turning circle test case for wPCC, time series from model test and simulation.
```


In [ ]:
dataframes_ = dataframes.copy()
dataframes_.update(dataframes_monte_carlo)
fig = plot(dataframes=dataframes_, keys=['u','v','r'], ncols=1, styles=styles, zero_origo=False);
ax = fig.axes[0]
ax.plot([],[],'b-',label='Monte Carlo MAVMM')
ax.legend()
fig.savefig("figures/fig_testing_sim.pdf")
glue("fig_testing_sim", fig, display=False)


In [ ]:
result_turning_circle = pd.DataFrame()
turning_circle = TurningCircle(angle=35, nominal_speed=df_test['U'].iloc[0], lpp=ship_data['L'], df=df_test)
result_turning_circle['True'] = pd.Series(turning_circle.evaluate(), name='True')
df_result = result_martin.result.copy()
df_result['V'] = np.sqrt(df_result['u']**2 + df_result['v']**2)
turning_circle = TurningCircle(angle=35, nominal_speed=df_test['U'].iloc[0], lpp=ship_data['L'], df=df_result)
result_turning_circle['Test Modified Abkowitz'] = pd.Series(turning_circle.evaluate(), name='Test Modified Abkowitz')

In [ ]:
result_turning_circle.drop(index='units',inplace=True)

In [ ]:
result_turning_circle.pct_change(axis=1)

In [ ]:
df_table = result_turning_circle.transpose()[['advance', 'Advance (IMO)','tactical_diameter', 'Tactical diameter (IMO)']]
df_table = df_table.astype(float)
df_table.rename(index={'True':'Model test', 'Test Modified Abkowitz':'Prediction'}, inplace=True)
df_table.rename(columns={'advance':'Advance [m]', 
                         'Advance (IMO)':'Advance (IMO) [m]',
                         'tactical_diameter':'Tactical diameter [m]', 
                         'Tactical diameter (IMO)':'Tactical diameter (IMO) [m]',
                        
                        }, inplace=True)

print(df_to_myst(df_table.round(decimals=2), 
                 title="wPCC Predicted turning circle advance and tactical diameter compared to SSPA model tests and IMO limit",
                 name="tab_wpcc_advance",
                 include_index=True)
                 )

```{list-table} wPCC Predicted turning circle advance and tactical diameter compared to SSPA model tests and IMO limit
:header-rows: 1
:name: tab_wpcc_advance
* -  
  - Advance [m]
  - Advance (IMO) [m]
  - Tactical diameter [m]
  - Tactical diameter (IMO) [m]
* - Model test
  - 12.82
  - 22.57
  - 14.76
  - 25.07
* - Prediction
  - 13.3
  - 22.57
  - 14.93
  - 25.07

```


In [ ]:
vmm_name = 'vmm_martins_simple'
regression = catalog.load(f"{ ship }.updated.{ vmm_name }.joined.regression")

models = {'X': regression.model_X,
          'Y': regression.model_Y,
          'N': regression.model_N}

mean_parameters_all = []
for dof, model in models.items():

    mean_parameters = pd.DataFrame()
    mean_parameters['mean'] = model.params*1000
    mean_parameters['se'] = model.bse*1000
    
    names = pd.Series(mean_parameters.index)
    mean_parameters.index = names.apply(parameter_to_latex)
    mean_parameters['name'] = mean_parameters.index
    mean_parameters.reset_index(inplace=True, drop=True)
    mean_parameters_all.append(mean_parameters[['name','mean','se']].copy())
    
mean_parameters = pd.concat(mean_parameters_all, axis=1) 
mean_parameters_table = mean_parameters.copy()
mean_parameters_table=np.around(mean_parameters_table, decimals=3)
mean_parameters_table.fillna('', inplace=True)

print(df_to_myst(mean_parameters_table, 
                 title=f'wPCC MAVMM derivatives (times 1000)', 
                 name=f"wpcc_derivatives", include_index=False))

In [ ]:
mean_parameters_table